# GST on Rigetti Backends with PyGSTi
The purpose of this notebook is to use the data collected from a GST experiment that has been read into the `rigetti_first_run` directory and perform GST on it. 

This code was mostly taken from the tutorial notebook [UnitaryFundDemoNotebook1.ipynb
](https://zenodo.org/record/5715199#.Yr_TOUjMLJF). 

In [602]:
#PyGSTi tools
import pygsti
from pygsti.modelpacks import smq1Q_XZ

#Mitiq tools
from mitiq.pec import NoisyOperation, NoisyBasis, execute_with_pec
from mitiq.pec.representations.optimal import find_optimal_representation, find_optimal_representation_super

from mitiq import QPROGRAM
from mitiq.interface import convert_to_mitiq
from mitiq.pec.types import NoisyBasis, OperationRepresentation
from mitiq.pec.channels import matrix_to_vector, kraus_to_super
from mitiq.pec.channels import choi_to_super, _circuit_to_choi

#Helper libraries
from typing import cast, List, Optional
import numpy as np
import numpy.typing as npt
from scipy.optimize import minimize, LinearConstraint

#make numpy matrices easier to read
np.set_printoptions(precision=3, suppress=True)

# Qiskit
import qiskit
from qiskit.quantum_info import PTM, SuperOp, Kraus, Pauli, Operator
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.test.mock import *
from qiskit.providers.fake_provider import *
from qiskit.providers.aer import AerSimulator, QasmSimulator, Aer
from qiskit.providers.aer.noise import phase_amplitude_damping_error
from qiskit.providers.aer.noise.noise_model import NoiseModel
from qiskit.quantum_info.operators.channel.transformations import *
from qiskit.quantum_info.operators.channel.quantum_channel import QuantumChannel
from qiskit.quantum_info.operators.channel.transformations import _choi_to_kraus, _kraus_to_superop
from qiskit.quantum_info.operators.channel.transformations import _kraus_to_choi, _choi_to_superop

# Cirq library used as a frontend for interfacing Mitiq
import cirq
from cirq import kraus

from qutip import *
from qutip.qip.operations import expand_operator
from qutip.steadystate import steadystate

In [614]:
z2 = expand_operator(sigmaz(), 3, 0) * expand_operator(sigmaz(), 3, 1) * expand_operator(sigmaz(), 3, 2)
x1 = expand_operator(sigmax(), 3, 0)
x2 = expand_operator(sigmax(), 3, 1)
x3 = expand_operator(sigmax(), 3, 2)

In [142]:
error = phase_amplitude_damping_error(0.001,0.001)
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, ['id'])

In [159]:
for i, j in enumerate(Qobj((SuperOp(error).tensor(SuperOp(np.identity(4))).data + SuperOp(np.identity(4)).tensor(SuperOp(error)).data)/2).full()):
    for k, l in enumerate(j):
        if l > 0:
            print(i, k, l)

0 0 (0.9999999999999997+0j)
0 5 (0.0004999999999999999+0j)
0 10 (0.0004999999999999999+0j)
1 1 (0.9994997497496867+0j)
1 11 (0.0004999999999999999+0j)
2 2 (0.9994997497496867+0j)
2 7 (0.0004999999999999999+0j)
3 3 (0.9989994994993737+0j)
4 4 (0.9994997497496867+0j)
4 14 (0.0004999999999999999+0j)
5 5 (0.9994999999999997+0j)
5 15 (0.0004999999999999999+0j)
6 6 (0.9989994994993737+0j)
7 7 (0.9989997497496867+0j)
8 8 (0.9994997497496867+0j)
8 13 (0.0004999999999999999+0j)
9 9 (0.9989994994993737+0j)
10 10 (0.9994999999999997+0j)
10 15 (0.0004999999999999999+0j)
11 11 (0.9989997497496867+0j)
12 12 (0.9989994994993737+0j)
13 13 (0.9989997497496867+0j)
14 14 (0.9989997497496867+0j)
15 15 (0.9989999999999998+0j)


In [156]:
Qobj(SuperOp(error).tensor(SuperOp(np.identity(4))).data)

Quantum object: dims = [[16], [16]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.001 0.
  0.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.001
  0.    0.    0.    0.   ]
 [0.    0.    0.999 0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.999 0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.001 0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.001]
 [0.    0.    0.    0.    0.    0.    0.999 0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.999 0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.999 0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    

In [162]:
for i, j in enumerate(Qobj(SuperOp(error).tensor(SuperOp(np.identity(4))).data).full()):
    for k, l in enumerate(j):
        if l > 0:
            print(i, k, l)

0 0 (0.9999999999999997+0j)
0 10 (0.0009999999999999998+0j)
1 1 (0.9999999999999997+0j)
1 11 (0.0009999999999999998+0j)
2 2 (0.9989994994993737+0j)
3 3 (0.9989994994993737+0j)
4 4 (0.9999999999999997+0j)
4 14 (0.0009999999999999998+0j)
5 5 (0.9999999999999997+0j)
5 15 (0.0009999999999999998+0j)
6 6 (0.9989994994993737+0j)
7 7 (0.9989994994993737+0j)
8 8 (0.9989994994993737+0j)
9 9 (0.9989994994993737+0j)
10 10 (0.9989999999999998+0j)
11 11 (0.9989999999999998+0j)
12 12 (0.9989994994993737+0j)
13 13 (0.9989994994993737+0j)
14 14 (0.9989999999999998+0j)
15 15 (0.9989999999999998+0j)


## Read data from experiment
This data was written by the `get_gst_data.ipynb` file.

In [2]:
# Usa data from first run
data = pygsti.io.read_data_from_dir('experiment_data/rigetti_first_run')

# Use data from second run
# data = pygsti.io.read_data_from_dir('experiment_data_all/second_experiment_run_data/experiment_data/rigetti_first_run')


## Run GST on the data

In this subsection we impose that RZ  must be equal to the ideal operation.

This is procedure is adapted from page 3 of https://arxiv.org/abs/2002.12476.


In [3]:
# Run GST assuming a perfect rz gate.
target_mdl = smq1Q_XZ.target_model()
rz_matrix = target_mdl[('Gzpi2',0)].to_dense()
dense_rz = pygsti.modelmembers.operations.DenseOperator(rz_matrix, evotype="default")
target_mdl[('Gzpi2',0)] = dense_rz.copy()

results = pygsti.protocols.GST(target_mdl, gaugeopt_suite=None).run(data)

--- Iterative GST: Iter 1 of 6  52 circuits ---: 
  MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
     1 atoms, parameter block size limits (None,)
  *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
      More atom-processors than hosts: each host gets ~1 atom-processors
      Atom-processors already occupy a single node, dividing atom-processor into 1 param-processors.
  *** Divided 1-host atom-processor (~1 procs) into 1 param-processing groups ***
  --- chi2 GST ---
  Sum of Chi^2 = 93.5413 (52 data params - 28 (approx) model params = expected mean of 24; p-value = 3.70919e-10)
  Completed in 0.2s
  Iteration 1 took 0.3s
  
--- Iterative GST: Iter 2 of 6  96 circuits ---: 
  MatrixLayout: 1 processors divided into 1 x 1 (= 1) grid along circuit and parameter directions.
     1 atoms, parameter block size limits (None,)
  *** Distributing 1 atoms to 1 atom-processing groups (1 cores) ***
      More atom-processors t

In [4]:
model = results.estimates['GateSetTomography'].models['final iteration estimate']
print("Rx Pauli transfer matrix:\n", model.operations[('Gxpi2', 0)].to_dense())
print("Rz Pauli transfer matrix:\n", model.operations[('Gzpi2', 0)].to_dense())

Rx Pauli transfer matrix:
 [[ 0.999  0.002 -0.011  0.007]
 [ 0.01   0.989  0.062 -0.064]
 [ 0.062 -0.057  0.012 -0.974]
 [ 0.06  -0.059  0.976  0.016]]
Rz Pauli transfer matrix:
 [[ 1.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]


## Get noisy superoperators

In [2]:
# Convert Pauli transfer matrix to superoperator
superop_rx = SuperOp(PTM(model.operations[('Gxpi2', 0)].to_dense())).data.conj()  # conj depends on convention
superop_rz = SuperOp(PTM(model.operations[('Gzpi2', 0)].to_dense())).data.conj()  # conj depends on convention



print("Rx superoperator matrix:\n", superop_rx)
print("Rz superoperator matrix:\n", superop_rz)


NameError: name 'model' is not defined

## Get ideal superoperators. Method 1.

In [2]:
qb = [cirq.LineQubit(x) for x in range(0, 2)]

In [3]:
qb[1]

cirq.LineQubit(1)

In [4]:
def Rzz(q0, q1, theta):
    yield cirq.CNOT(q0, q1)
    yield cirq.rz(theta).on(q1)
    yield cirq.CNOT(q0, q1)
    
def Ryy(q0, q1, theta):
    yield [cirq.ZPowGate(exponent=0.5).on(q0),cirq.ZPowGate(exponent=0.5).on(q1)]
    yield [cirq.H(q0), cirq.H(q1)]
    yield cirq.CNOT(q0, q1)
    yield cirq.rz(theta).on(q1)
    yield cirq.CNOT(q0, q1)
    yield [cirq.HPowGate(exponent=-1).on(q0), cirq.HPowGate(exponent=-1).on(q1)]
    yield [cirq.ZPowGate(exponent=-0.5).on(q0),cirq.ZPowGate(exponent=-0.5).on(q1)]
    
def Rxx(q0, q1, theta):
    yield [cirq.H(q0), cirq.H(q1)]
    yield cirq.CNOT(q0, q1)
    yield cirq.rz(theta).on(q1)
    yield cirq.CNOT(q0, q1)
    yield [cirq.HPowGate(exponent=-1).on(q0), cirq.HPowGate(exponent=-1).on(q1)]

In [575]:
dt = 0.005*np.pi/2
thet = -1 * dt

# For 1 qubit
circuitZ = cirq.Circuit()
circuitZ.append(cirq.rz(thet).on(qb[0]))

circuitX = cirq.Circuit()
circuitX.append(cirq.rx(thet).on(qb[0]))

# Create ideal operations as Cirq circuits
circuitrXI = cirq.Circuit()
circuitrXI.append(cirq.rx(thet).on(qb[0]))
circuitrXI.append(cirq.I(qb[1]))
circuitIrX = cirq.Circuit()
circuitIrX.append(cirq.rx(thet).on(qb[1]))
circuitIrX.append(cirq.I(qb[0]))
circuitrZI = cirq.Circuit()
circuitrZI.append(cirq.rz(thet).on(qb[0]))
circuitrZI.append(cirq.I(qb[1]))
circuitIrZ = cirq.Circuit()
circuitIrZ.append(cirq.rz(thet).on(qb[1]))
circuitIrZ.append(cirq.I(qb[0]))

circuitXI = cirq.Circuit()
circuitXI.append(cirq.X(qb[0]))
circuitXI.append(cirq.I(qb[1]))
circuitIX = cirq.Circuit()
circuitIX.append(cirq.X(qb[1]))
circuitIX.append(cirq.I(qb[0]))

circuitsXI = cirq.Circuit()
circuitsXI.append(cirq.XPowGate(exponent=0.5).on(qb[0]))
circuitsXI.append(cirq.I(qb[1]))
circuitIsX = cirq.Circuit()
circuitIsX.append(cirq.XPowGate(exponent=0.5).on(qb[1]))
circuitIsX.append(cirq.I(qb[0]))

circuitZZ = cirq.Circuit()
cirq_rzz = circuitZZ.append(Rzz(qb[0], qb[1], thet))
# cirq_rzz = cirq.Circuit(cirq.rzz(np.pi / 2).on(cirq.LineQubit(0)))
circuitXX = cirq.Circuit()
cirq_rxx = circuitXX.append(Rxx(qb[0], qb[1], thet))
# cirq_rxx = cirq.Circuit(cirq.rx(np.pi / 2).on(cirq.LineQubit(0)))
circuitYY = cirq.Circuit()
cirq_ryy = circuitYY.append(Ryy(qb[0], qb[1], thet))

In [576]:
circuitZZ

0: ───@─────────────────@───
      │                 │
1: ───X───Rz(-0.002π)───X───

In [552]:
from mitiq.pec.channels import choi_to_super, _circuit_to_choi

test_ZZ = choi_to_super(_circuit_to_choi(circuitZZ))

In [553]:
from mitiq.pec.channels import choi_to_super, _circuit_to_choi

superop_rxx_mitiq = choi_to_super(_circuit_to_choi(circuitXX))
superop_ryy_mitiq = choi_to_super(_circuit_to_choi(circuitYY))
superop_rzz_mitiq = choi_to_super(_circuit_to_choi(circuitZZ))
# print(superop_rxx_mitiq)
# print(superop_ryy_mitiq)
superop_rxx_mitiq

array([[1.+0.j   , 0.+0.j   , 0.+0.j   , 0.-0.008j, 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.008j, 0.+0.j   , 0.+0.j   , 0.+0.j   ],
       [0.+0.j   , 1.+0.j   , 0.-0.008j, 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.008j, 0.+0.j   , 0.+0.j   ],
       [0.+0.j   , 0.-0.008j, 1.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.008j, 0.+0.j   ],
       [0.-0.008j, 0.+0.j   , 0.+0.j   , 1.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.008j],
       [0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j   , 1.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.-0.008j, 0.+0.008j, 0.+0.j   , 0.+0.j   , 0.+0.j   ,
        0.+0.j   , 0.+0.j   , 0.+0.j   , 0.+0.j 

## Get superoperators. Method 2

In [528]:
superop_rxx_cirq = cirq.kraus_to_superoperator(cirq.kraus(circuitXX))
superop_rzz_cirq = cirq.kraus_to_superoperator(cirq.kraus(circuitZZ))
print(superop_rxx_cirq)
print(superop_rzz_cirq)

[[0.998+0.j    0.   +0.j    0.   +0.j    0.   -0.039j 0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.039j 0.   +0.j    0.   +0.j
  0.002+0.j   ]
 [0.   +0.j    0.998+0.j    0.   -0.039j 0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.039j 0.002+0.j
  0.   +0.j   ]
 [0.   +0.j    0.   -0.039j 0.998+0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.002+0.j    0.   +0.039j
  0.   +0.j   ]
 [0.   -0.039j 0.   +0.j    0.   +0.j    0.998+0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.002+0.j    0.   +0.j    0.   +0.j
  0.   +0.039j]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.998+0.j
  0.   +0.j    0.   +0.j    0.   -0.039j 0.   +0.039j 0.   +0.j
  0.   +0.j    0.002+0.j    0.   +0.j

## Get superoperators from QuTiP (qiskit too)

In [577]:
## This is the \sigma_+ \sigma_- part:
n_qubits = 2
al = 1
L = 0
for n in range(n_qubits):
    c_op_left = expand_operator(sigmap(), n_qubits, n)
    c_op_right = expand_operator(sigmam(), n_qubits, n)
    L += al*sprepost(c_op_left, c_op_right)
    both = -0.5 * al * c_op_right * c_op_left
    L += spre(both)
    L += spost(both)

In [578]:
L

Quantum object: dims = [[[2, 2], [2, 2]], [[2, 2], [2, 2]]], shape = (16, 16), type = super, isherm = False
Qobj data =
[[ 0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.   0.   0.   0.
   0.   0. ]
 [ 0.  -0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.
   0.   0. ]
 [ 0.   0.  -0.5  0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.  -0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.
   1.   0. ]
 [ 0.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   1. ]
 [ 0.   0.   0.   0.   0.   0.  -1.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -1.5  0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -0.5  0.   0.   0.   0.   1.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.  -1.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.

In [579]:
ga = 0.003; t = 1
ell = Qobj(ga*t*L).expm()
ell

Quantum object: dims = [[[2, 2], [2, 2]], [[2, 2], [2, 2]]], shape = (16, 16), type = super, isherm = False
Qobj data =
[[1.    0.    0.    0.    0.    0.003 0.    0.    0.    0.    0.003 0.
  0.    0.    0.    0.   ]
 [0.    0.999 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.003
  0.    0.    0.    0.   ]
 [0.    0.    0.999 0.    0.    0.    0.    0.003 0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.997 0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.999 0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.003 0.   ]
 [0.    0.    0.    0.    0.    0.997 0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.003]
 [0.    0.    0.    0.    0.    0.    0.997 0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.996 0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.999 0.    0.    0.
  0.    0.003 0.    0.

In [580]:
## This is the \sigma_+ \sigma_- part:
nk = 2
lz = 0
al = 1
be = 1
for n in range(nk):
    c_op_left = expand_operator(sigmap(), nk, n)
    c_op_right = expand_operator(sigmam(), nk, n)
    z_op_left = expand_operator(sigmaz(), nk, n)
    z_op_right = expand_operator(sigmaz(), nk, n)
    lz += al * sprepost(c_op_left, c_op_right)
    lz += be * sprepost(z_op_left, z_op_right)
    both = -0.5 * al * c_op_right * c_op_left
    bothz = -0.5 * be * z_op_right * z_op_left
    lz += spre(both)
    lz += spost(both)
    lz += spre(bothz)
    lz += spost(bothz)

In [581]:
de = 0.001; t = 1
L_AD = Qobj(de*t*lz).expm()
# L_AD

In [582]:
## 1 qubit stuff
qcrz = QuantumCircuit(1)
qcrz.rz(thet, 0)
qcrx = QuantumCircuit(1)
qcrx.rx(thet, 0)

qcx = QuantumCircuit(1)
qcx.x(0)

qcsx = QuantumCircuit(1)
qcsx.sx(0)

## 2 qubit stuff
# 1 qubit gates on 2 qubits
qcrzi = QuantumCircuit(2)
qcrzi.rz(thet, 0)
qcirz = QuantumCircuit(2)
qcirz.rz(thet, 1)
qcrxi = QuantumCircuit(2)
qcrxi.rx(thet, 0)
qcirx = QuantumCircuit(2)
qcirx.rx(thet, 1)

qcxi = QuantumCircuit(2)
qcxi.x(0)
qcix = QuantumCircuit(2)
qcix.x(1)

qcsxi = QuantumCircuit(2)
qcsxi.sx(0)
qcisx = QuantumCircuit(2)
qcisx.sx(1)

# the 2 qubit gates to simulate
qczz = QuantumCircuit(2)
qczz.rzz(thet, 0, 1)
qcxx = QuantumCircuit(2)
qcxx.rxx(thet, 0, 1)
qcyy = QuantumCircuit(2)
qcyy.ryy(thet, 0, 1)

qccx = QuantumCircuit(2)
qccx.cx(0, 1)

In [583]:
rzSupOp = Qobj(SuperOp(qcrz).data).tidyup(atol=1e-10).full().conj()
rxSupOp = Qobj(SuperOp(qcrx).data).tidyup(atol=1e-10).full().conj()

In [332]:
noisy_rz = rzSupOp @ ell.full()
noisy_rx = rxSupOp @ ell.full()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 16 is different from 4)

In [584]:
rrziSupOp = Qobj(SuperOp(qcrzi).data).tidyup(atol=1e-10).full().conj()
rrxiSupOp = Qobj(SuperOp(qcrxi).data).tidyup(atol=1e-10).full().conj()
rirzSupOp = Qobj(SuperOp(qcirz).data).tidyup(atol=1e-10).full().conj()
rirxSupOp = Qobj(SuperOp(qcirx).data).tidyup(atol=1e-10).full().conj()
rxiSupOp = Qobj(SuperOp(qcxi).data).tidyup(atol=1e-10).full().conj()
rixSupOp = Qobj(SuperOp(qcix).data).tidyup(atol=1e-10).full().conj()
rsxiSupOp = Qobj(SuperOp(qcsxi).data).tidyup(atol=1e-10).full().conj()
risxSupOp = Qobj(SuperOp(qcisx).data).tidyup(atol=1e-10).full().conj()

In [585]:
noisy_rzi = rrziSupOp @ ell.full()
noisy_riz = rirzSupOp @ ell.full()
noisy_rxi = rrxiSupOp @ ell.full()
noisy_rix = rirxSupOp @ ell.full()
noisy_xi = rxiSupOp @ ell.full()
noisy_ix = rixSupOp @ ell.full()
noisy_sxi = rsxiSupOp @ ell.full()
noisy_isx = risxSupOp @ ell.full()

In [586]:
noisy_rzi_ad = rrziSupOp @ L_AD.full()
noisy_riz_ad = rirzSupOp @ L_AD.full()
noisy_rxi_ad = rrxiSupOp @ L_AD.full()
noisy_rix_ad = rirxSupOp @ L_AD.full()
noisy_xi_ad = rxiSupOp @ L_AD.full()
noisy_ix_ad = rixSupOp @ L_AD.full()
noisy_sxi_ad = rsxiSupOp @ L_AD.full()
noisy_isx_ad = risxSupOp @ L_AD.full()

In [587]:
rzzSupOp = Qobj(SuperOp(qczz).data).tidyup(atol=1e-10).full().conj()
rxxSupOp = Qobj(SuperOp(qcxx).data).tidyup(atol=1e-10).full().conj()
ryySupOp = Qobj(SuperOp(qcyy).data).tidyup(atol=1e-10).full().conj()

In [588]:
noisy_rzz = rzzSupOp @ ell.full()
noisy_rxx = rxxSupOp @ ell.full()
noisy_ryy = ryySupOp @ ell.full()

In [589]:
noisy_rzz_ad = rzzSupOp @ L_AD.full()
noisy_rxx_ad = rxxSupOp @ L_AD.full()
noisy_ryy_ad = ryySupOp @ L_AD.full()

In [590]:
Qobj(noisy_rzz)

Quantum object: dims = [[16], [16]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[1.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.003+0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.003+0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.998+0.008j 0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.003+0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.998+0.008j 0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.003+0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.997+0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.998-0.008j
  0.  

In [591]:
Qobj(noisy_rzz_ad)

Quantum object: dims = [[16], [16]], shape = (16, 16), type = oper, isherm = False
Qobj data =
[[1.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.001+0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.001+0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.997+0.008j 0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.001+0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.997+0.008j 0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.001+0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.995+0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j
  0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j    0.997-0.008j
  0.  

## Define the basis of noisy operations

In [592]:
# Create noisy operations as Mitiq NoisyOperation objects for 1 qubit
# mitiq_noisy_rx = NoisyOperation(circuitX, noisy_rx)
# mitiq_noisy_rz = NoisyOperation(circuitZ, noisy_rz)

# Combine into basis object, 1 qubit
# base_noisy_basis = NoisyBasis(mitiq_noisy_rx, mitiq_noisy_rz)

# Create noisy operations as Mitiq NoisyOperation objects for 2 qubits
# mitiq_noisy_rrxi = NoisyOperation(circuitrXI, noisy_rxi_ad)
# mitiq_noisy_rirx = NoisyOperation(circuitIrX, noisy_rix_ad)
# mitiq_noisy_rrzi = NoisyOperation(circuitrZI, noisy_rzi_ad)
mitiq_noisy_rrzi = NoisyOperation(circuitrZI, rrziSupOp)

# mitiq_noisy_rirz = NoisyOperation(circuitIrZ, noisy_riz_ad)
mitiq_noisy_rirz = NoisyOperation(circuitIrZ, rirzSupOp)

mitiq_noisy_rxx = NoisyOperation(circuitXX, noisy_rxx_ad)
mitiq_noisy_ryy = NoisyOperation(circuitYY, noisy_ryy_ad)
mitiq_noisy_rzz = NoisyOperation(circuitZZ, noisy_rzz_ad)

mitiq_noisy_xi = NoisyOperation(circuitXI, noisy_xi_ad)
mitiq_noisy_ix = NoisyOperation(circuitIX, noisy_ix_ad)
mitiq_noisy_sxi = NoisyOperation(circuitsXI, noisy_sxi_ad)
mitiq_noisy_isx = NoisyOperation(circuitIsX, noisy_isx_ad)

# Combine into basis object, 2 qubits: mitiq_noisy_rrxi, mitiq_noisy_rirx,
base_noisy_basis = NoisyBasis(mitiq_noisy_rrzi, mitiq_noisy_rirz, mitiq_noisy_xi, 
                              mitiq_noisy_ix, mitiq_noisy_sxi, mitiq_noisy_isx,
                              mitiq_noisy_rzz)

# Extend basis to squences of noisy gates
noisy_basis = NoisyBasis(
    *base_noisy_basis.get_sequences(1),
    *base_noisy_basis.get_sequences(2),
#     *base_noisy_basis.get_sequences(3),
#     *base_noisy_basis.get_sequences(4),
#     *base_noisy_basis.get_sequences(5),
#     *base_noisy_basis.get_sequences(6),
#     *base_noisy_basis.get_sequences(7),
)

print("Sequeces of noisy operations to be used as a basis for PEC:")
for elem in noisy_basis.elements:
    print(elem)

Sequeces of noisy operations to be used as a basis for PEC:
0: ───Rz(-0.002π)───I───────

1: ───I─────────────X^0.5───
0: ───I─────────────X───

1: ───Rz(-0.002π)───I───
0: ───Rz(-0.002π)───X^0.5───

1: ───I─────────────I───────
0: ───X───I───

1: ───I───X───
0: ───X───I─────────────

1: ───I───Rz(-0.002π)───
0: ───Rz(-0.002π)───

1: ───I─────────────
0: ───I───Rz(-0.002π)───

1: ───X───I─────────────
0: ───X^0.5───Rz(-0.002π)───

1: ───I───────I─────────────
0: ───@─────────────────@───I───
      │                 │
1: ───X───Rz(-0.002π)───X───X───
0: ───X^0.5───

1: ───I───────
0: ───@─────────────────@───@─────────────────@───
      │                 │   │                 │
1: ───X───Rz(-0.002π)───X───X───Rz(-0.002π)───X───
0: ───Rz(-0.002π)───I─────────────

1: ───I─────────────Rz(-0.002π)───
0: ───I───────@─────────────────@───
              │                 │
1: ───X^0.5───X───Rz(-0.002π)───X───
0: ───I───────I───

1: ───X^0.5───X───
0: ───I─────────────

1: ───Rz(-0.002π)───
0:

In [522]:
mitiq_noisy_rzz.circuit('cirq')

0: ───@────────────────@───
      │                │
1: ───X───Rz(-0.05π)───X───

## Compute optimal representation

In [267]:
superop_rzz_cirq

array([[1.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.707+0.707j, 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.707+0.707j, 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ],
       [0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 1.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0.   +0.j   , 0.   +0.j   ,
        0.   +0.j   , 0.   +0.j   , 0

In [599]:
# Get optimal representation in terms of noisy gates
# optimal_rep = find_optimal_representation(circuitZZ, noisy_basis, tol=.001) #circuitZZ
optimal_rep = find_optimal_representation_super(circuitZZ, noisy_rzz, noisy_basis, tol=.002)

print(optimal_rep)
print("Overhead: ", optimal_rep.norm) # Hopefully close to 1

0: ───@─────────────────@───
      │                 │
1: ───X───Rz(-0.002π)───X─── = 

0.125
0: ───Rz(-0.002π)───

1: ───I─────────────

-0.122
0: ───@─────────────────@───I───
      │                 │
1: ───X───Rz(-0.002π)───X───X───

+0.002
0: ───@─────────────────@───@─────────────────@───
      │                 │   │                 │
1: ───X───Rz(-0.002π)───X───X───Rz(-0.002π)───X───

+0.001
0: ───Rz(-0.002π)───I─────────────

1: ───I─────────────Rz(-0.002π)───

+0.125
0: ───I─────────────

1: ───Rz(-0.002π)───

+0.122
0: ───I───@─────────────────@───
          │                 │
1: ───X───X───Rz(-0.002π)───X───

+0.001
0: ───I─────────────Rz(-0.002π)───

1: ───Rz(-0.002π)───I─────────────

-0.125
0: ───X───X───

1: ───I───I───

-0.122
0: ───@─────────────────@───X───
      │                 │
1: ───X───Rz(-0.002π)───X───I───

+0.992
0: ───@─────────────────@───
      │                 │
1: ───X───Rz(-0.002π)───X───

-0.125
0: ───I───I───

1: ───X───X───

+0.122
0: ───X───@───

In [597]:
optimal_rep.noisy_operations[1].circuit()

0: ───I─────────────X───

1: ───Rz(-0.002π)───I───

In [ ]:
self._ideal, self._native_type = convert_to_mitiq(ideal)

In [414]:
optimal_rep.noisy_operations[0].circuit('qiskit').draw()

┌───┐ 
q_0: ┤ I ├─
     ├───┴┐
q_1: ┤ √X ├
     └────┘

In [416]:
len(optimal_rep.noisy_operations)

90

In [492]:
sum([abs(m) for m in optimal_rep.coeffs])

2.06608168257626

In [417]:
# Compute PEC linear combination
matrices = [m.channel_matrix for m in optimal_rep.noisy_operations]
mat = sum(m*coeff for (m,coeff) in zip(matrices, optimal_rep.coeffs))
print("Approximated superoperator")
print(mat)
print('------------------------------')
print("Ideal superoperator")
print(noisy_rzz)

Approximated superoperator
[[ 0.999+0.j     0.001+0.001j  0.001+0.001j -0.   +0.j     0.001-0.001j
   0.003+0.j     0.   +0.j     0.   +0.j     0.001-0.001j  0.   +0.j
   0.003+0.j     0.   +0.j    -0.   +0.j     0.   -0.j     0.   -0.j
  -0.   +0.j   ]
 [ 0.001+0.001j  0.921+0.381j -0.   +0.j     0.001+0.001j -0.   +0.001j
  -0.001-0.001j  0.   +0.j     0.   +0.j     0.   +0.j    -0.001-0.001j
   0.   +0.j     0.002+0.001j  0.   -0.j    -0.   +0.j    -0.   +0.j
  -0.   -0.j   ]
 [ 0.001+0.001j -0.   +0.j     0.921+0.381j  0.001+0.001j  0.   +0.j
   0.   +0.j    -0.001-0.001j  0.002+0.001j -0.   +0.001j  0.   +0.j
  -0.001-0.001j  0.   +0.j     0.   -0.j    -0.   +0.j    -0.   +0.j
  -0.   -0.j   ]
 [-0.   +0.j     0.001+0.001j  0.001+0.001j  0.996-0.001j  0.   +0.j
   0.   +0.j     0.001+0.j     0.001-0.001j  0.   +0.j     0.001+0.j
   0.   +0.j     0.001-0.001j -0.   +0.j     0.   -0.j     0.   -0.j
  -0.   +0.j   ]
 [ 0.001-0.001j -0.   -0.001j  0.   +0.j     0.   +0.j     0.921-0.3

In [418]:
print(np.max(mat - noisy_rzz))
# print(np.max(superop_rzz_cirq - noisy_rzz))

(0.001000003166350738+0j)


## Define noisy executor

In [373]:
def ideal_to_noisy_circuit(ideal_circuit):
    noisy_rx = cirq.KrausChannel(cirq.superoperator_to_kraus(superop_rx)).on(cirq.LineQubit(0))
    noisy_rz = cirq.KrausChannel(cirq.superoperator_to_kraus(superop_rz)).on(cirq.LineQubit(0))

    noisy_circuit = cirq.Circuit()
    for op in ideal_circuit.all_operations():
        if op.gate == cirq.rx(np.pi / 2):
            noisy_circuit.append(noisy_rx)
        elif op.gate == cirq.rz(np.pi / 2):
            noisy_circuit.append(noisy_rz)
        else:
            print("Error:", op)

    return noisy_circuit

In [194]:
import pygsti

def cirq_to_pygsti(ideal_circuit: cirq.Circuit):
    pygsti_ops = []
    for op in ideal_circuit.all_operations(): 
        if op.gate == cirq.rx(np.pi / 2):
            pygsti_ops.append(('Gxpi2',0))
        elif op.gate == cirq.rz(np.pi / 2):
            pygsti_ops.append(('Gzpi2',0))
        else:
            print("Error: impossible to simulate operation", op)

    return pygsti.circuits.Circuit(pygsti_ops, line_labels=[0])

In [195]:
def cirq_executor(circuit):
    """Returns the expectation value of the observable A = |1><1|"""
    noisy_circuit = ideal_to_noisy_circuit(circuit)
    rho = cirq.DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[1, 1].real

def pygsti_executor(circuit):
    """Returns the expectation value of the observable A = |1><1|"""
    noisy_circuit = cirq_to_pygsti(circuit)
    probs = model.probabilities(noisy_circuit)
    return probs["1"]

## Define circuit to mitigate

In [35]:
bit_flip_circuit = cirq_rx + cirq_rx

## Execute without error mitigation

In [46]:
# cirq_executor(bit_flip_circuit)
pygsti_executor(bit_flip_circuit)

0.8278870446802166

## Apply PEC

In [57]:
# execute_with_pec(bit_flip_circuit, cirq_executor, representations=(optimal_rep, ), random_state=0, num_samples=5000)

In [48]:
execute_with_pec(bit_flip_circuit, pygsti_executor, representations=(optimal_rep, ), random_state=0, num_samples=5000)

4.843116547296679

In [495]:
# Modification to the Mitiq function:

def find_optimal_representation_super(
    ideal_operation: QPROGRAM,
    noisy_super_operator: npt.NDArray[np.complex64],
    noisy_basis: NoisyBasis,
    tol: float = 1.0e-8,
    initial_guess: Optional[npt.NDArray[np.float64]] = None,
) -> OperationRepresentation:
    r"""Returns the ``OperationRepresentaiton`` of the input ideal operation
    which minimizes the one-norm of the associated quasi-probability
    distribution.

    More precicely, it solve the following optimization problem:

    .. math::
        \min_{{\eta_\alpha}} = \sum_\alpha |\eta_\alpha|,
        \text{ such that }
        \mathcal G = \sum_\alpha \eta_\alpha \mathcal O_\alpha,

    where :math:`\{\mathcal O_j\}` is the input basis of noisy operations.

    Args:
        ideal_operation: The circuit element to approximate, as a QPROGRAM.
        noisy_super_operator: The noisy super operator to approximate; i.e.,
            the (A) circuit to approximate in terms of (B) circuits, and input
            as a npt.NDArray[np.complex64]
        noisy_basis: The ``NoisyBasis`` in which the ``ideal_operation``
            should be represented. It must contain ``NoisyOperation`` objects
            which are initialized with a numerical superoperator matrix.
        tol: The error tolerance for each matrix element
            of the represented operation.
        initial_guess: Optional initial guess for the coefficients
            :math:`\{ \eta_\alpha \}``.

    Returns: The optimal OperationRepresentation.
    """
    # ideal_cirq_circuit, _ = convert_to_mitiq(ideal_operation)
    # ideal_matrix = kraus_to_super(
    #     cast(List[npt.NDArray[np.complex64]], kraus(ideal_cirq_circuit))
    # )
    ideal_matrix = noisy_super_operator
    basis_set = noisy_basis.elements

    try:
        basis_matrices = [noisy_op.channel_matrix for noisy_op in basis_set]
    except ValueError as err:
        if str(err) == "The channel matrix is unknown.":
            raise ValueError(
                "The input noisy_basis should contain NoisyOperation objects"
                " which are initialized with a numerical superoperator matrix."
            )
        else:
            raise err  # pragma no cover

    # Run numerical optimization problem
    quasi_prob_dist = minimize_one_norm(
        ideal_matrix,
        basis_matrices,
        tol=tol,
        initial_guess=initial_guess,
    )

    basis_expansion = {op: eta for op, eta in zip(basis_set, quasi_prob_dist)}
    return OperationRepresentation(ideal_operation, basis_expansion)